In [2]:
%reload_ext ishbook
import pandas as pd
import datetime as dt
import plus
import iql
import numpy as np
from pytz import timezone
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
## Read in the master sales list and ONLY grab the reps that have been marked as "Current Employee" to get only the active reps on sales teams

sales_sheet = plus.read_google_sheet('1dTuz8gPQ8d1REKJ2wwoif34QczcfhkzDOhaW6ubkEJc',intype='key',sheet='Sales Roster')
current_sales_rep = sales_sheet['Current Employee'] == 'YES'
current_sales_sheet = sales_sheet[current_sales_rep]

In [16]:
current_sales_sheet.head()

,Name,New Team Names,Market,Title,Title,Current/Final Director,Current/Final Sr. Director,Current Office Location,Current Region,Past Office Location,...,HR/Indeed Teams Name,Indeed Assigned Name,QTR Name Change,Year Name Change,Notes,Top Gun 2016 Purposes (KP),,Market Old,Hire Date (m/d/y),Term Date (m/d/y)
ID,,,,,,,,,,,,,,,,,,,,,
21,Stan Berizzi,Agency-US-Northeast,US,Agency AM,SENIOR AGENCY ACCOUNT EXECUTIVE,Amanda Colonna,Michael Nasif,Remote,US/CA,,...,,sberizzi,,,Title updated 3/26/2015,Y,,Agency US,5/22/2006,
47,Santo Provenzano,NYC ALL,NYC,Senior Director NYC,SENIOR DIRECTOR OF SALES,,Santo Provenzano,NYC,US/CA,,...,,santo,,,,Y,,NYC,6/19/2007,
66,Joe Sgrulletta,US ALL,US,VP US Inside Sales,Vice President Inside Sales US,,,STM,US/CA,,...,,joes,,,,Y,,US,2/19/2008,
102,Amanda Colonna,Agency-US-East,Agency US,Agency Director Agency-US-East,DIRECTOR OF SALES Agency Relations,Amanda Colonna,Michael Nasif,Remote,US/CA,,...,,acolonna,,,,Y,,Agency US,9/22/2008,
101,Nolan Farris,ALL,ALL,SVP Sales,Senior Vice President Sales,,,STM,ALL,,...,,nolan,,,,Y,,ALL,9/27/2008,


In [17]:
## remove the blank values from the User ID field - this is what I'm going to use to create a list of IDs to feed into the index

remove_blanks = current_sales_sheet['User ID'] != ""
current_sales_sheet = current_sales_sheet[remove_blanks]
current_sales_sheet['User ID'] = current_sales_sheet['User ID'].astype(int)

In [18]:
## Since the User ID field comes in as a string object, I have to reassign each value in the column as integer in order to feed into the index
## this is why I had to drop the blank values in the previous cell - Python can't convert an empty string into an integer
## create a list of IDs & refine the sales_sheet so it only includes the fields that I want

sales_ids = tuple(current_sales_sheet['User ID'].tolist())
current_sales_sheet = current_sales_sheet[['Name', 'New Team Names', 'User ID']]

In [19]:
## Do the same steps as above for the CS Team list, pull in the current CS Reps

cs_teams = plus.read_google_sheet('1194g28N02sqVJMhHHQthdKItgzgbwWwaAfgC5YVVFko', intype = 'key', sheet = 'Current Team Info')

In [20]:
## Except in this step, I'm removing the "Non-Client Facing" reps/teams - this gets rid of a lot of the senior mgmt and teams that don't touch clients

remove_non_client_facing = cs_teams['Category'] != 'Non-Client Facing'
cs_teams = cs_teams[remove_non_client_facing]

In [21]:
## Same as above, filtering for only the columns that I need

cs_teams_filtered = cs_teams[['Common Name', 'Team', 'AdCentral ID']].copy()
cs_teams_filtered.reset_index(inplace = True)

In [22]:
drop_na = cs_teams_filtered['AdCentral ID'] != '#N/A'
cs_teams_filtered = cs_teams_filtered[drop_na]
cs_teams_filtered['AdCentral ID'] = cs_teams_filtered['AdCentral ID'].astype(int)

In [23]:
cs_ids = tuple(cs_teams_filtered['AdCentral ID'].tolist())

In [24]:
current_sales_sheet.tail(1)
current_sales_sheet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1322 entries, 21 to 6068
Data columns (total 3 columns):
Name              1322 non-null object
New Team Names    1322 non-null object
User ID           1322 non-null int64
dtypes: int64(1), object(2)
memory usage: 41.3+ KB


In [25]:
cs_teams_filtered.tail(1)

,,Common Name,Team,AdCentral ID
684,,Zuzana Kovacova,CS-EMEA-UK-CSStaffingJB,6041


In [27]:
## for all the rep_ids associated to sales reps, go into the 'advertiserevent' index and grab any activity_types for 'CREATE_AD' & 'CREATE_JOB' for Hosted jobs
## grab the unixtime so I can correlate this to a date

%ish internal_jobs_sales = from advertiserevent 2016-05-01 2016-09-30 \
where rep_id in {sales_ids} activity_type in ('CREATE_AD', 'CREATE_JOB') activity_field=HOSTED_JOB \
group by rep_id, rep_username/*Sales Rep*/, advertiser_id, message, unixtime select counts/*Number of Created Jobs*/

Output = None

In [28]:
internal_jobs_sales.head(20)

,rep_id,Sales Rep,advertiser_id,message,unixtime,Number of Created Jobs
0,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
1,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
2,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
3,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
4,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
5,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
6,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098436,1
7,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098444,1
8,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098473,1
9,3409,manuels,6215939,[claudiavvc@mx.inditex.com] Created hosted job...,1462098492,1


In [29]:
test = internal_jobs_sales.copy()

In [30]:
test['Date (EST)'] = [dt.datetime.fromtimestamp(x).replace(tzinfo=timezone('EST')).strftime('%F') for x in test.unixtime]
test['Time (EST)'] = [dt.datetime.fromtimestamp(x).replace(tzinfo=timezone('EST')).strftime('%I:%M:%S %p') for x in test.unixtime]

In [35]:
test2 = test.groupby(['Date (EST)','Time (EST)', 'Sales Rep', 'rep_id', 'advertiser_id']).agg({'Number of Created Jobs': sum}).reset_index()

In [32]:
test2[test2['Number of Created Jobs'] == 274]

,Date (EST),Time (EST),Sales Rep,advertiser_id,Number of Created Jobs
4052,2016-09-06,04:41:18 AM,acahill,1041163,274


In [41]:
test2.merge(current_sales_sheet, how = 'left', left_on = 'rep_id', right_on = 'User ID')

,Date (EST),Time (EST),Sales Rep,rep_id,advertiser_id,Number of Created Jobs,Name,New Team Names,User ID
0,2016-05-01,06:26:55 AM,manuels,3409,6215939,6,Manuel Sanz,ES-IS-DUB,3409
1,2016-05-01,06:27:16 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
2,2016-05-01,06:27:24 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
3,2016-05-01,06:27:53 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
4,2016-05-01,06:28:12 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
5,2016-05-01,06:28:29 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
6,2016-05-01,06:28:50 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
7,2016-05-01,06:29:05 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
8,2016-05-01,06:29:16 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409
9,2016-05-01,06:29:26 AM,manuels,3409,6215939,1,Manuel Sanz,ES-IS-DUB,3409


In [38]:
current_sales_sheet

,Name,New Team Names,User ID
ID,,,
21,Stan Berizzi,Agency-US-Northeast,21
47,Santo Provenzano,NYC ALL,47
66,Joe Sgrulletta,US ALL,66
102,Amanda Colonna,Agency-US-East,102
101,Nolan Farris,ALL,101
138,Quinn Smith,,138
137,Brendan Pomeroy,STM 2 ALL,137
149,Aline Eastwick-Field,Sales Support,149
166,Ricky Fritsch,NationalAccounts-East-DE,166


In [57]:
internal_jobs_sales['Date'] = [dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d').replace(tzinfo=timezone('EST')) for x in internal_jobs_sales.unixtime]

TypeError: replace() takes no keyword arguments

In [65]:
## do the same query for CS Reps

%ish internal_jobs_cs = from advertiserevent 2016-05-01 2016-09-30 \
where rep_id in ({cs_ids}) activity_type in ('CREATE_AD', 'CREATE_JOB') activity_field=HOSTED_JOB \
group by rep_id, rep_username/*Sales Rep*/, advertiser_id, activity_type, activity_field, unixtime select counts/*Number of Created Jobs*/

Output = None

In [85]:
internal_jobs_cs.tail(10)

,rep_id,Sales Rep,advertiser_id,activity_type,activity_field,unixtime,Number of Created Jobs,Date
10149,5776,aurelie,171145,CREATE_AD,HOSTED_JOB,1475184233,1,2016-09-29
10150,5026,elizabethl,1813192,CREATE_AD,HOSTED_JOB,1475186191,1,2016-09-29
10151,2364,gbogan,1211061,CREATE_AD,HOSTED_JOB,1475187032,5,2016-09-29
10152,4312,mika,8535857,CREATE_AD,HOSTED_JOB,1475202551,1,2016-09-29
10153,4312,mika,8535857,CREATE_AD,HOSTED_JOB,1475202587,1,2016-09-29
10154,4312,mika,6564191,CREATE_AD,HOSTED_JOB,1475203467,2,2016-09-29
10155,4311,chie,8086118,CREATE_AD,HOSTED_JOB,1475211614,1,2016-09-30
10156,4311,chie,8086118,CREATE_AD,HOSTED_JOB,1475211622,1,2016-09-30
10157,4311,chie,8086118,CREATE_AD,HOSTED_JOB,1475211630,1,2016-09-30
10158,3662,tomokom,5419383,CREATE_AD,HOSTED_JOB,1475215183,1,2016-09-30


In [69]:
## create a date field based on the unix time - I'm going to keep the unixtime for my next query down below

internal_jobs_sales['Date'] = [dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d') for x in internal_jobs_sales.unixtime]
internal_jobs_cs['Date'] = [dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d') for x in internal_jobs_cs.unixtime]

In [77]:
## organize the data frame by grouping the dates, advertisers, reps
## we'll use this organized df to sync it up with the teams we pulled in earlier

sales_grouped = internal_jobs_sales.groupby(['Date', 'advertiser_id', 'rep_id', 'Sales Rep', 
                            'activity_field']).agg({'Number of Created Jobs' : sum}).reset_index()

In [80]:
sales_group_teams = sales_grouped.merge(current_sales_sheet, how = 'left', left_on = 'rep_id', right_on = 'User ID')

In [101]:
sales_group_teams.tail(1)

,Date,advertiser_id,rep_id,Sales Rep,activity_field,Number of Created Jobs,Name,New Team Names,User ID
2821,2016-09-29,9200491,2841,asclama,HOSTED_JOB,1,Andrew Sclama,NYC-DE3,2841


In [84]:
sales_group_teams.groupby(['Date', 'advertiser_id', 'New Team Names', 'Name', 'activity_field']).agg({'Number of Created Jobs' : sum}).to_clipboard()

In [89]:
## do the same organization for CS - it seems redundant to do this in separate frames but it's way cleaner to just have two frames and two outputs
## when I first ran through this, it took FOREVER because I tried to split them up after doing all this manipulation

cs_grouped = internal_jobs_cs.groupby(['Date', 'advertiser_id', 'rep_id', 'activity_field']).agg({'Number of Created Jobs' : sum}).reset_index()

In [91]:
cs_group_teams = cs_grouped.merge(cs_teams_filtered, how = 'left', left_on = 'rep_id', right_on = 'AdCentral ID')

In [94]:
cs_group_teams.tail(1)

,Date,advertiser_id,rep_id,activity_field,Number of Created Jobs,,Common Name,Team,AdCentral ID
6703,2016-09-30,8086118,4311,HOSTED_JOB,3,,Chie Furukawa,CSJapan,4311


In [131]:
cs_group_teams.groupby(['Date', 'advertiser_id', 'Team', 'Common Name', 'activity_field']).agg({'Number of Created Jobs' : sum}).reset_index().to_clipboard()

In [ ]:
## second part / component to this request
## we need to grab the ip addresses & instances where a rep logged into an advertiser's account
## and tie this back to the above data sets

In [4]:
%ish ip_address = from advertiserevent 2016-05-01 2016-09-30 \
where activity_type=LOGIN message=~".*(1.79.81.67|10.1.5.147|101.180.40.131|104.132.11.102|104.162.196.170|104.188.155.24|106.161.214.20|108.58.248.74|113.155.32.232|115.110.171.106|115.110.171.107|115.248.210.154|121.244.117.246|121.244.154.37|121.244.63.107|121.244.63.108|126.152.13.151|126.152.195.240|126.188.91.213|126.236.200.246|126.254.134.247|126.255.195.217|14.203.216.101|148.75.164.155|153.214.169.206|173.227.56.162|173.54.173.153|174.196.128.78|176.67.168.190|183.74.201.101|185.122.133.162|185.122.133.165|198.58.79.20|202.213.202.148|203.174.133.60|207.114.131.162|207.191.44.146|208.90.215.138|209.203.71.74|209.66.80.122|210.142.96.144|210.142.99.67|213.61.155.114|213.61.223.198|216.123.6.194|216.136.25.66|217.110.59.147|219.96.24.157|24.27.227.128|24.4.4.35|24.72.250.132|27.110.45.75|27.110.45.76|32.210.151.9|32.211.128.176|32.213.23.228|36.12.60.194|37.228.226.234|37.228.235.193|38.88.242.122|49.181.174.220|63.143.194.245|65.156.210.234|66.108.171.63|67.180.210.172|67.250.82.28|67.79.201.170|67.79.229.226|67.87.151.186|67.87.184.142|68.117.92.152|68.198.146.79|68.9.155.169|70.115.133.210|70.197.13.6|70.209.21.101|70.215.14.36|70.215.17.6|73.38.223.42|74.73.82.219|75.64.145.184|80.113.207.5|80.133.138.221|80.169.36.138|82.141.220.200|84.14.148.211|89.101.23.145|97.105.47.162).*" message!=~".*(@indeed.com|@indeedagency|via ).*" \
group by rep_id, advertiser_id, message, activity_type, unixtime

Output = None

In [10]:
len(ip_address)

4168

In [11]:
ip_address

,rep_id,advertiser_id,message,activity_type,unixtime,
0,0,1228212,[Yasmin@indeed.Com] Logged in from 203.174.133...,LOGIN,1462152661,1
1,0,3711173,[simona@responseconcepts.nl] Logged in from 89...,LOGIN,1462169805,1
2,0,6657629,[u.pfersich@hpp.de] Logged in from 198.58.79.2...,LOGIN,1462173085,1
3,0,6452691,[anthony.leat@centralfinances.fr] Logged in fr...,LOGIN,1462174511,1
4,0,6452691,[anthony.leat@centralfinances.fr] Logged in fr...,LOGIN,1462177162,1
5,0,6988678,[renault@4ventsgroup.com] Logged in from 198.5...,LOGIN,1462178395,1
6,0,3654324,[alex.urbals@web.de] Logged in from 198.58.79....,LOGIN,1462193125,1
7,0,7021268,[husemann@onapply.de] Logged in from 198.58.79...,LOGIN,1462194854,1
8,0,2801930,[imeoconseil@precontact.fr] Logged in from 198...,LOGIN,1462194946,1
9,0,7021268,[husemann@onapply.de] Logged in from 198.58.79...,LOGIN,1462195788,1


In [123]:
internal_jobs_cs.merge(ip_address, how = 'inner', on = 'advertiser_id')#.groupby(['Date', 'advertiser_id', 'Sales Rep', 'message']).agg({'Number of Created Jobs' : sum})

,rep_id,Sales Rep,advertiser_id,activity_type,activity_field,unixtime_x,Number of Created Jobs,Date,unixtime_y,message,
0,2332,nana,2540837,CREATE_AD,HOSTED_JOB,1462173446,1,2016-05-02,1463734990,[recruit@aands-inc.co.jp] Logged in from 202.2...,1
1,2332,nana,2540837,CREATE_AD,HOSTED_JOB,1462173446,1,2016-05-02,1465203198,[recruit@aands-inc.co.jp] Logged in from 202.2...,1
2,2885,alienor,5769320,CREATE_AD,HOSTED_JOB,1462173618,22,2016-05-02,1464602257,[recrutement.causeaeffet@gmail.com] Logged in ...,1
3,2889,sashar,2811295,CREATE_JOB,HOSTED_JOB,1462248720,1,2016-05-03,1462248904,Logged in as SubAccount from 203.174.133.60 [M...,1
4,2889,sashar,2811295,CREATE_JOB,HOSTED_JOB,1462248720,1,2016-05-03,1462256969,Logged in as SubAccount from 203.174.133.60 [M...,1
5,3644,aisling,4585031,CREATE_AD,HOSTED_JOB,1462273504,1,2016-05-03,1474379945,[jamie.talbot@planetcaval.com] Logged in from ...,1
6,4417,maridawn,2266999,CREATE_AD,HOSTED_JOB,1462286869,4,2016-05-03,1464305167,[a@ring.com] Logged in from 207.114.131.162 [u...,1
7,4417,maridawn,2266999,CREATE_AD,HOSTED_JOB,1462286918,1,2016-05-03,1464305167,[a@ring.com] Logged in from 207.114.131.162 [u...,1
8,4417,maridawn,2266999,CREATE_AD,HOSTED_JOB,1462286964,1,2016-05-03,1464305167,[a@ring.com] Logged in from 207.114.131.162 [u...,1
9,4417,maridawn,2266999,CREATE_AD,HOSTED_JOB,1462286999,1,2016-05-03,1464305167,[a@ring.com] Logged in from 207.114.131.162 [u...,1


In [26]:
%ish sales_update_ads = from advertiserevent 2016-05-01 2016-09-30 \
where rep_id in {sales_ids} activity_type in ('UPDATE_AD_BUDGET', 'UPDATE_AD_BID') activity_field = 'HOSTED_JOB' \
group by rep_id/*User ID*/, rep_username, advertiser_id, message, unixtime

Output = None

In [27]:
sales_update_ads2 = sales_update_ads.copy()

In [28]:
sales_update_ads2['Date (EST)'] = [dt.datetime.fromtimestamp(x).replace(tzinfo=timezone('EST')).strftime('%F') for x in sales_update_ads2.unixtime]
sales_update_ads2['Time (EST)'] = [dt.datetime.fromtimestamp(x).replace(tzinfo=timezone('EST')).strftime('%I:%M:%S %p') for x in sales_update_ads2.unixtime]

In [29]:
sales_update_ads2

,User ID,rep_username,advertiser_id,message,unixtime,,Date (EST),Time (EST)
0,3409,manuels,4294817,[seleccion@grconsultoria.mx] Raised monthly bu...,1462098069,1,2016-05-01,06:21:09 AM
1,3409,manuels,4294817,[seleccion@grconsultoria.mx] Raised monthly bu...,1462098069,1,2016-05-01,06:21:09 AM
2,3409,manuels,4294817,[seleccion@grconsultoria.mx] Raised monthly bu...,1462098069,1,2016-05-01,06:21:09 AM
3,3409,manuels,4294817,[seleccion@grconsultoria.mx] Raised monthly bu...,1462098069,1,2016-05-01,06:21:09 AM
4,3409,manuels,641090,[aperalta@cmr.mx] Lowered monthly budget for h...,1462098110,1,2016-05-01,06:21:50 AM
5,3409,manuels,4579034,[sasoto@sears.com.mx] Raised monthly budget fo...,1462098197,1,2016-05-01,06:23:17 AM
6,3409,manuels,4579034,[sasoto@sears.com.mx] Raised monthly budget fo...,1462098209,1,2016-05-01,06:23:29 AM
7,3409,manuels,4579034,[sasoto@sears.com.mx] Raised monthly budget fo...,1462098221,1,2016-05-01,06:23:41 AM
8,3409,manuels,4579034,[sasoto@sears.com.mx] Raised monthly budget fo...,1462098236,1,2016-05-01,06:23:56 AM
9,3409,manuels,4579034,[sasoto@sears.com.mx] Raised monthly budget fo...,1462098250,1,2016-05-01,06:24:10 AM


In [30]:
test = sales_update_ads2.groupby(['Date (EST)', 'User ID', 'advertiser_id', 'message']).agg({'unixtime' : 'count'}).reset_index()

In [34]:
test.merge(current_sales_sheet, how = 'left' , on = 'User ID').to_csv('sales_bids.csv', encoding = 'UTF-8')

In [73]:
current_sales_sheet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1322 entries, 21 to 6068
Data columns (total 3 columns):
Name              1322 non-null object
New Team Names    1322 non-null object
User ID           1322 non-null object
dtypes: object(3)
memory usage: 41.3+ KB


In [ ]:
### TESTING AREA

In [125]:

%ish test_df = from advertiserevent 2016-05-01 2016-09-30 \
where rep_id in ({sales_ids}) activity_type in ('CREATE_AD', 'CREATE_JOB') activity_field=HOSTED_JOB \
group by rep_id, rep_username/*Sales Rep*/, advertiser_id, activity_type, activity_field, message, unixtime select counts/*Number of Created Jobs*/

Output = None

In [130]:
test_df

,rep_id,Sales Rep,advertiser_id,activity_type,activity_field,message,unixtime,Number of Created Jobs
0,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
1,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
2,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
3,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
4,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
5,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098415,1
6,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098436,1
7,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098444,1
8,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098473,1
9,3409,manuels,6215939,CREATE_AD,HOSTED_JOB,[claudiavvc@mx.inditex.com] Created hosted job...,1462098492,1
